In [35]:
from gates import *
from assembler import HackAssembler

In [89]:
class SRNandLatch:
    def __init__(self):
        self.q = 0
        self.not_q = 1

    def eval(self, r, s):
        if s == 0:
            self.not_q = Nand(self.q, s)
            self.q = Nand(r, self.not_q)
        elif r == 0:
            self.q = Nand(r, self.not_q)
            self.not_q = Nand(self.q, s)

        return self
    
    def __repr__(self):
        return f'q={self.q}, not_q={self.not_q}'

In [ ]:
class CPU:
    def __init__(self):
        self.a_register = Register()
        self.d_register = Register()
        self.pc = PC()
        self.prev_alu_out = (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    
    def eval(self, instruction, in_m, reset):
        
        # 1xxaccccccdddjjj

        a_instruction = Not(instruction[15])
        c_instruction = Not(a_instruction)

        a = instruction[12]
        zx = instruction[11]
        nx = instruction[10]
        zy = instruction[9]
        ny = instruction[8]
        f = instruction[7]
        no = instruction[6]
        write_a = Or(a_instruction, instruction[5])
        write_d = And(c_instruction, instruction[4])
        write_m = And(c_instruction, instruction[3])
        lt = instruction[2]
        eq = instruction[1]
        gt = instruction[0]

        which_instruction = Mux16(a=self.prev_alu_out, b=instruction,
                                  sel=a_instruction) 

        a_register_out = self.a_register.eval(which_instruction,
                                              load=write_a)

        a_or_m = Mux16(a_register_out, in_m, a)

        d_register_out = self.d_register.eval(self.prev_alu_out, load=write_d)

        alu_out, zr, ng = ALU(d_register_out, a_or_m,
                              zx, nx, zy, ny, f, no)

        jump = Or(Or(And(lt, ng),
                     And(eq, zr)),
                  And(gt, And(Not(ng),
                              Not(zr))))
        
        pc_out = self.pc(inp=a_register_out, load=jump, inc=Not(jump), reset=reset)

        self.prev_alu_out = alu_out

        return alu_out, write_m, a_register_out, pc_out

In [36]:
assembler = HackAssembler()
assembler.translate('Fill.asm')

In [38]:
help(HackAssembler)

Help on class HackAssembler in module assembler:

class HackAssembler(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  translate(self, asm_path)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables
 |  
 |  __weakref__
 |      list of weak references to the object

